# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,14.33,74,0,3.94,ZA,1741666978
1,1,pacasmayo,-7.4006,-79.5714,23.50,76,99,5.16,PE,1741666979
2,2,ishpeming,46.4885,-87.6676,9.52,82,82,6.20,US,1741666980
3,3,tolanaro,-25.0319,46.9987,25.04,88,40,4.12,MG,1741666981
4,4,port-aux-francais,-49.3500,70.2167,9.18,92,99,18.07,TF,1741666983


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',       
    y='Lat',        
    geo=True,
    tiles='OSM',    # OpenStreetMap tiles
    size='Humidity', # Size of the points based on humidity
    color='City',    # Color points by city
    frame_width=800,
    frame_height=600
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[(city_data_df['Max Temp'] >= 30) & (city_data_df['Humidity'] <= 50)]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna(subset=['Lat', 'Lng', 'Humidity', 'Max Temp'])

# Display sample data
filtered_cities.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,pyapon,16.2833,95.6833,35.35,31,9,3.27,MM,1741667003
29,29,geraldton,-28.7667,114.6000,32.23,31,0,3.09,AU,1741667012
65,65,mount isa,-20.7333,139.5000,37.87,32,75,2.06,AU,1741667039
91,91,codrington,-38.2667,141.9667,35.05,27,81,2.30,AU,1741667083
117,117,howrah,22.5892,88.3103,30.00,39,0,4.12,IN,1741667113
229,229,mangrol,21.1167,70.1167,30.34,36,0,2.67,IN,1741667241
278,278,khetia,21.6667,74.5833,31.78,20,1,2.48,IN,1741667297
304,304,kununurra,-15.7667,128.7333,34.99,49,94,4.63,AU,1741667328
305,305,razole,16.4833,81.8333,31.00,41,79,2.55,IN,1741667329
345,345,alice springs,-23.7000,133.8833,37.75,13,68,7.20,AU,1741667375


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
22,pyapon,MM,16.2833,95.6833,31,
29,geraldton,AU,-28.7667,114.6000,31,
65,mount isa,AU,-20.7333,139.5000,32,
91,codrington,AU,-38.2667,141.9667,27,
117,howrah,IN,22.5892,88.3103,39,
229,mangrol,IN,21.1167,70.1167,36,
278,khetia,IN,21.6667,74.5833,20,
304,kununurra,AU,-15.7667,128.7333,49,
305,razole,IN,16.4833,81.8333,41,
345,alice springs,AU,-23.7000,133.8833,13,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",  # Specify the category for hotels
    "apiKey": geoapify_key,                # Your Geoapify API key
    "limit": 1                              # Limit the results to the nearest hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pyapon - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Hotel
mount isa - nearest hotel: Ibis Styles
codrington - nearest hotel: No hotel found
howrah - nearest hotel: Sun India Guest House
mangrol - nearest hotel: No hotel found
khetia - nearest hotel: No hotel found
kununurra - nearest hotel: No hotel found
razole - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
suong - nearest hotel: No hotel found
porbandar - nearest hotel: Toran Tourist Bungalow
cox's bazar - nearest hotel: Hotel Taj Sheba
dampit - nearest hotel: No hotel found
mount gambier - nearest hotel: The Old Mount Gambier Gaol
lubao - nearest hotel: Hacienda de Gracia Resort
ashta - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
noamundi - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
22,pyapon,MM,16.2833,95.6833,31,No hotel found
29,geraldton,AU,-28.7667,114.6000,31,Ocean Centre Hotel
65,mount isa,AU,-20.7333,139.5000,32,Ibis Styles
91,codrington,AU,-38.2667,141.9667,27,No hotel found
117,howrah,IN,22.5892,88.3103,39,Sun India Guest House
229,mangrol,IN,21.1167,70.1167,36,No hotel found
278,khetia,IN,21.6667,74.5833,20,No hotel found
304,kununurra,AU,-15.7667,128.7333,49,No hotel found
305,razole,IN,16.4833,81.8333,41,No hotel found
345,alice springs,AU,-23.7000,133.8833,13,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng',                   
    y='Lat',                   
    geo=True,                  
    tiles='OSM',              
    size='Humidity',           # Size of the points based on humidity
    color='Hotel Name',        # Color points by hotel name
    frame_width=800,           
    frame_height=600,          
    hover_cols=['City', 'Country', 'Hotel Name']  # Add hover information
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,City,Country)